# Import 

In [1]:
import sklearn_crfsuite
from sklearn_crfsuite.metrics import flat_f1_score
from sklearn_crfsuite import CRF
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from spacy.lang.en import English
from spacy import displacy
from pathlib import Path
import joblib
from spacy.tokens import Doc
from typing import List
import random
from statistics import mean, stdev
from loguru import logger
import sys
import spacy

In [2]:
from medspacy_io.reader.brat_reader import BratDocReader, BratDirReader
import medspacy

## Initiate BratDirReader

In [3]:
cleaned_train_dir=r'..\data\n2c2\cleaned_training'
cleaned_test_dir=r'..\data\n2c2\cleaned_test'
Path(cleaned_train_dir).exists(), Path(cleaned_test_dir).exists()

(True, True)

In [4]:
nlp=spacy.load('en_core_web_sm', disable=['ner'])

In [5]:
dir_reader = BratDirReader(nlp=nlp, schema_file=str(Path(cleaned_train_dir, 'annotation.conf')), support_overlap=True)

## Read Brat annotations | load from pickles

In [6]:
pickle_file= r'..\data\n2c2\spacy_docs.joblib'

In [7]:
if not Path(pickle_file).exists():
    train_docs=dir_reader.read(txt_dir=cleaned_train_dir)
    test_docs=dir_reader.read(txt_dir=cleaned_test_dir)
    print(len(train_docs), len(test_docs))
    joblib.dump((train_docs, test_docs), pickle_file)
else:
    print(f'{pickle_file} already exists, load them directly')
    # before load from pickle, initiate EhostDirReader or EhostDocReader first, because some Doc extension used to store meta data will not be automatically recreated by loading.
    train_docs, test_docs=joblib.load(pickle_file)

..\data\n2c2\spacy_docs.joblib already exists, load them directly


# Define sampling functions

In [8]:
rounds =10
seed= 14

In [9]:
len(train_docs), len(test_docs)

(303, 202)

### Define CRF Wrapper

In [10]:
from CRFWrapper import spans_to_bio, convert_docs, word2features, sent2features,compute_metrics_and_averages,  CRFModel
from ALSampler import SamplingSimulator, ModelSamplingSimulator

In [11]:
## Get all annotation types: 
annos=set()
for d in train_docs:
    for anno in d.spans.keys():
        annos.add(anno)
print(annos)

{'Form', 'Drug', 'Route', 'Dosage', 'Strength', 'ADE', 'Duration', 'Frequency', 'Reason'}


In [12]:
crf_model=CRFModel(anno_types=annos)
m_simulator=ModelSamplingSimulator(train_docs, total_round=10, modelWrapper=crf_model, eval_docs=test_docs, init_seed=seed)

### Test run

In [13]:
scores=m_simulator.simulate_rounds()

2024-04-04 22:37:20.776 | INFO     | ALSampler:simulate_rounds:45 - simulate round 0.
2024-04-04 22:37:20.777 | DEBUG    | ALSampler:keep_sample:35 - The first round sampling will be random
2024-04-04 22:37:20.779 | INFO     | ALSampler:keep_sample:39 - current sampled: 30, remaining: 273
2024-04-04 22:37:24.462 | DEBUG    | CRFWrapper:fit:251 - Reset and train CRF model...
2024-04-04 22:38:20.944 | DEBUG    | CRFWrapper:fit:259 - Training complete.
2024-04-04 22:38:21.041 | DEBUG    | CRFWrapper:bootstrap_eval:280 - Predicting eval docs...
2024-04-04 22:38:57.564 | DEBUG    | CRFWrapper:bootstrap_eval:284 - Calculate scores from bootstrapping 200 times
2024-04-04 22:43:11.749 | DEBUG    | CRFWrapper:bootstrap_eval:293 - complete
2024-04-04 22:43:11.882 | DEBUG    | ALSampler:simulate_rounds:56 - {'precision': 0.9435403132345506, 'recall': 0.7792371047764345, 'f1': 0.8535492442836575}
2024-04-04 22:43:11.884 | INFO     | ALSampler:simulate_rounds:45 - simulate round 1.
2024-04-04 22:43

In [14]:
def compute_mean_ci(scores):
    ave=np.mean(scores)
    ci=np.percentile(scores, [2.5, 97.5])
    return ave, ci

summary={'precision': [], 'pl':[], 'pu': [], 'recall': [], 'rl':[], 'ru': [], 'f1':[], 'fl':[], 'fu': []}
for s in scores:    
    for k,v in s.items():
        ave, (l, u)=compute_mean_ci(v)
        summary[k].append(ave)
        summary[k[0]+'l'].append(l)
        summary[k[0]+'u'].append(u)


In [15]:
pd.options.display.float_format='{:,.4f}'.format

In [16]:
pd.DataFrame(summary)

,precision,pl,pu,recall,rl,ru,f1,fl,fu
0,0.9435,0.9402,0.9472,0.7792,0.7715,0.7860,0.8535,0.8485,0.8582
1,0.9455,0.9419,0.9489,0.8093,0.8024,0.8170,0.8721,0.8671,0.8764
2,0.9451,0.9419,0.9487,0.8272,0.8222,0.8340,0.8822,0.8790,0.8869
3,0.9461,0.9429,0.9494,0.8374,0.8318,0.8432,0.8884,0.8846,0.8923
4,0.9448,0.9408,0.9482,0.8441,0.8384,0.8496,0.8916,0.8879,0.8954
5,0.9444,0.9413,0.9474,0.8527,0.8476,0.8582,0.8962,0.8929,0.8998
6,0.9472,0.9443,0.9501,0.8528,0.8477,0.8579,0.8975,0.8941,0.9011
7,0.9452,0.9421,0.9481,0.8574,0.8526,0.8629,0.8991,0.8956,0.9024
8,0.9481,0.9451,0.9518,0.8571,0.8530,0.8616,0.9003,0.8977,0.9036
9,0.9458,0.9423,0.9490,0.8595,0.8545,0.8641,0.9006,0.8967,0.9040


## Bootstrap 3 different initials runs

In [17]:
boostrap_runs=3
total_round=10

In [18]:
logger.remove()
logger.add(sys.stderr, level='INFO')

1

In [19]:
random.seed(14)
seeds=[random.randint(1,10000000) for  _ in range(boostrap_runs)]
seeds

[1792286, 8843471, 4142887]

In [20]:
all_scores=[]
for si, seed  in enumerate(seeds):
    logger.info(f'start run {si}.')
    crf_model=CRFModel(anno_types=annos)
    m_simulator=ModelSamplingSimulator(train_docs, total_round=total_round, modelWrapper=crf_model, eval_docs=test_docs, init_seed=seed)
    scores=m_simulator.simulate_rounds()
    all_scores.append(scores) 

2024-04-09 10:30:04.634 | INFO     | __main__:<module>:3 - start run 0.
2024-04-09 10:30:04.646 | INFO     | ALSampler:simulate_rounds:45 - simulate round 0.
2024-04-09 10:30:04.647 | INFO     | ALSampler:keep_sample:39 - current sampled: 30, remaining: 273
2024-04-09 10:35:57.078 | INFO     | ALSampler:simulate_rounds:45 - simulate round 1.
2024-04-09 10:37:11.900 | INFO     | ALSampler:keep_sample:39 - current sampled: 60, remaining: 243
2024-04-09 10:44:22.287 | INFO     | ALSampler:simulate_rounds:45 - simulate round 2.
2024-04-09 10:45:25.429 | INFO     | ALSampler:keep_sample:39 - current sampled: 90, remaining: 213
2024-04-09 10:53:49.786 | INFO     | ALSampler:simulate_rounds:45 - simulate round 3.
2024-04-09 10:54:43.363 | INFO     | ALSampler:keep_sample:39 - current sampled: 120, remaining: 183
2024-04-09 11:04:13.952 | INFO     | ALSampler:simulate_rounds:45 - simulate round 4.
2024-04-09 11:04:58.732 | INFO     | ALSampler:keep_sample:39 - current sampled: 150, remaining: 

In [17]:
joblib.dump(all_scores, r'..\data\vb_sampling\ner_crf_scores.joblib')

['..\\data\\vb_sampling\\ner_crf_scores.joblib']